In [1]:

from matching.algorithms import galeshapley
import operator
import random
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import multiprocess as mp 


simulation_results = pd.DataFrame({
    'n':[],
    'sigma_fraction':[],
    'delta_fraction':[]
})


In [2]:
n = 500
sigma_fraction = .01
runs = 100

sigma = sigma_fraction * n

In [3]:
print(runs)
print(n)




100
500


In [48]:

#define simulation function
def GS_Sim(
    n ,
    sigma_fraction 
):
    #make randomly assigned preferences
    Mpref = {}
    Wpref = {}
    for i in range(0, n):
        utility = []
        for x in range(0,n):
            id = 'W' + str(x)
            utility = utility + [(id, random.normalvariate(x, sigma))]
        utility.sort(key=operator.itemgetter(1))

        ranklist = []
        for x in utility:
            name = [x[0]]
            ranklist = ranklist + name
        id = 'M' + str(i)
        Mpref.update({id:ranklist})

    #copypasta from above
    for i in range(0, n):
        utility = []
        for x in range(0,n):
            id = 'M' + str(x)
            utility = utility + [(id, random.normalvariate(x, sigma))]
        utility.sort(key=operator.itemgetter(1))

        ranklist = []
        for x in utility:
            name = [x[0]]
            ranklist = ranklist + name
        id = 'W' + str(i)
        Wpref.update({id:ranklist})

    #Run matching
    M_propose= galeshapley(Mpref, Wpref)
    W_propose = galeshapley(Wpref, Mpref)

    #transform matchings into same-format list of tuples
    M_propose = list(M_propose.items())
    W_propose = [(v, k) for k, v in W_propose.items()]

    #sort matchings
    M_propose.sort(key=operator.itemgetter(0))
    W_propose.sort(key=operator.itemgetter(0))
    M_propose
    #Find % differences
    differences = (np.array(M_propose) == np.array(W_propose))

    percent_diff = np.average(differences, axis = 0).item(1)
    
    #next_result =  pd.DataFrame({
    #    'n':[n],
    #    'sigma_fraction':[sigma_fraction],
    #    'delta_fraction':[percent_diff]
    #})
    
    next_result = (n, sigma_fraction, percent_diff)
    return next_result
    

def simulate(x):
    sizes = [10, 50]
    sigmas = [.1, .01]
    result = []
    for n in sizes:
        for sigma_fraction in sigmas:
            result =result + [GS_Sim(n, sigma_fraction)]
    return result
    

    
with mp.Pool(8) as p:
    results = p.map(simulate, range(50))
list(results)


#with mp.Pool(8) as p:
#    results = p.map(GS_Sim, [50,50],[.01,.01])



[[(10, 0.1, 0.8), (10, 0.01, 1.0), (50, 0.1, 0.96), (50, 0.01, 0.8)],
 [(10, 0.1, 1.0), (10, 0.01, 0.5), (50, 0.1, 0.88), (50, 0.01, 0.96)],
 [(10, 0.1, 0.4), (10, 0.01, 1.0), (50, 0.1, 1.0), (50, 0.01, 0.9)],
 [(10, 0.1, 0.8), (10, 0.01, 0.6), (50, 0.1, 0.96), (50, 0.01, 1.0)],
 [(10, 0.1, 1.0), (10, 0.01, 0.5), (50, 0.1, 0.96), (50, 0.01, 0.96)],
 [(10, 0.1, 1.0), (10, 0.01, 0.8), (50, 0.1, 1.0), (50, 0.01, 1.0)],
 [(10, 0.1, 1.0), (10, 0.01, 1.0), (50, 0.1, 0.9), (50, 0.01, 0.94)],
 [(10, 0.1, 0.7), (10, 0.01, 0.7), (50, 0.1, 0.96), (50, 0.01, 1.0)],
 [(10, 0.1, 1.0), (10, 0.01, 0.5), (50, 0.1, 0.92), (50, 0.01, 0.88)],
 [(10, 0.1, 0.8), (10, 0.01, 1.0), (50, 0.1, 1.0), (50, 0.01, 0.92)],
 [(10, 0.1, 1.0), (10, 0.01, 0.7), (50, 0.1, 0.94), (50, 0.01, 0.8)],
 [(10, 0.1, 1.0), (10, 0.01, 1.0), (50, 0.1, 0.96), (50, 0.01, 1.0)],
 [(10, 0.1, 1.0), (10, 0.01, 0.6), (50, 0.1, 0.9), (50, 0.01, 0.86)],
 [(10, 0.1, 0.6), (10, 0.01, 0.6), (50, 0.1, 1.0), (50, 0.01, 0.8)],
 [(10, 0.1, 1.0), (1

In [10]:
def xsquare(x):
    return(x, x*x)
with mp.Pool(8) as p:
    results = p.map(xsquare, [1, 2, 3])

#results = map(GS_Sim, [1,2])
print(results)
list(results)

[(1, 1), (2, 4), (3, 9)]


[(1, 1), (2, 4), (3, 9)]

In [ ]:

results = map(multiply2, [1, 2, 3, 4]) 
print(results)
results

def f(real):
    return('bob')


mp.Pool(8)
results = mp.Pool.map(multiply2,[1, 2, 3, 4]) 
mp.Pool(close)
print(results)

In [6]:
def f(x):
    return x*x
def multiply2(x):
    res = ( x, x*2)
    return res
    
with mp.Pool(5) as p:
    print(p.map(multiply2, [1, 2, 3]))
    


[(1, 2), (2, 4), (3, 6)]
